In [5]:
import random
import numpy as np
from typing import List, Tuple

class Schedul:
    def __init__(self, population_size: int = 50, generations: int = 50):
        self.population_size = population_size
        self.generations = generations
        self.activities = {
            'A': 2,
            'B': 3,
            'C': 2,
            'D': 4
        }
        self.activity_list = ['A', 'B', 'C', 'D']
        self.show_details = True

    def create_individual(self) -> List[int]:
        return [random.randint(0, 7) for _ in range(4)]

    def create_population(self) -> List[List[int]]:
        return [self.create_individual() for _ in range(self.population_size)]

    def check_constraints(self, individual: List[int]) -> List[str]:
        violations = []

        # Ketentuan 1 : A harus sebelum B
        if individual[0] + self.activities['A'] > individual[1]:
            violations.append("A harus sebelum B")

        # Ketentuan 2 : C dan D tidak boleh overlap
        c_start, c_end = individual[2], individual[2] + self.activities['C']
        d_start, d_end = individual[3], individual[3] + self.activities['D']
        if not (c_end <= d_start or d_end <= c_start):
            violations.append("C dan D overlap")

        # Ketentuan 3 : Total durasi <= 8 jam
        end_times = [individual[i] + self.activities[act] for i, act in enumerate(self.activity_list)]
        if max(end_times) > 8:
            violations.append("Melebihi 8 jam")

        # Ketentuan 4 : Tidak ada overlap antar aktivitas
        activities = [(act, individual[i], individual[i] + self.activities[act])
                      for i, act in enumerate(self.activity_list)]

        for i in range(len(activities)):
            for j in range(i + 1, len(activities)):
                act1, start1, end1 = activities[i]
                act2, start2, end2 = activities[j]
                if not (end1 <= start2 or end2 <= start1):
                    violations.append(f"{act1} dan {act2} overlap")

        return violations

    def calculate_fitness(self, individual: List[int]) -> float:
        violations = self.check_constraints(individual)
        return max(0, 10 - len(violations))

    def get_sequence(self, individual: List[int]) -> List[str]:
        sorted_activities = sorted(enumerate(individual), key=lambda x: x[1])
        return [self.activity_list[idx] for idx, _ in sorted_activities]

    def format_schedule(self, individual: List[int]) -> str:
        return " | ".join([f"{act}:{time}" for act, time in zip(self.activity_list, individual)])

    def select_parents(self, population: List[List[int]], fitness_scores: List[float]) -> Tuple[List[int], List[int]]:
        tournament_size = 3
        parent1 = max(random.sample(list(enumerate(population)), tournament_size),
                      key=lambda x: fitness_scores[x[0]])[1]
        parent2 = max(random.sample(list(enumerate(population)), tournament_size),
                      key=lambda x: fitness_scores[x[0]])[1]
        return parent1, parent2

    def run(self) -> Tuple[List[int], float]:
        population = self.create_population()


        print("Format: [Waktu mulai A, B, C, D] -> Fitness")
        initial_fitness = [self.calculate_fitness(ind) for ind in population]

        print("\nMenampilkan 5 individu pertama dari", self.population_size, "individu")
        for i in range(min(5, self.population_size)):
            schedule = self.format_schedule(population[i])
            sequence = self.get_sequence(population[i])
            print(f"Individu {i + 1:2d}: {population[i]} -> Fitness: {initial_fitness[i]}")
            print(f"          Jadwal: {schedule}")
            print(f"          Urutan: {sequence}")
            if self.show_details:
                violations = self.check_constraints(population[i])
                if violations:
                    print("           Pelanggaran:", ", ".join(violations))
                else:
                    print("           Tidak ada pelanggaran")

        print(f"\nStatistik Populasi Awal:")
        print(f"Rata-rata Fitness: {np.mean(initial_fitness):.2f}")
        print(f"Fitness Tertinggi: {np.max(initial_fitness):.2f}")
        print(f"Fitness Terendah: {np.min(initial_fitness):.2f}")

        best_solution = None
        best_fitness = 0

        print("\n== Evaluasi fitness ===")
        for generation in range(self.generations):
            print(f"\nGENERASI {generation + 1:3d}/{self.generations}")
            print("-" * 10)

            # Evaluasi fitness
            fitness_scores = [self.calculate_fitness(ind) for ind in population]
            current_best_idx = fitness_scores.index(max(fitness_scores))
            current_best = population[current_best_idx]
            current_best_fitness = fitness_scores[current_best_idx]

            # Update best solution
            if current_best_fitness > best_fitness:
                best_fitness = current_best_fitness
                best_solution = current_best.copy()
                print("\n Solusi baru ditemukan!")
                print(f"Kromosom: {best_solution}")
                print(f"Jadwal  : {self.format_schedule(best_solution)}")
                print(f"Urutan  : {self.get_sequence(best_solution)}")
                print(f"Fitness : {best_fitness:.2f}")
                violations = self.check_constraints(best_solution)
                if violations:
                    print("Pelanggaran:", ", ".join(violations))
                else:
                    print("Tidak ada pelanggaran")

            # Buat populasi baru
            new_population = []

            # Elitism
            elite_idx = fitness_scores.index(max(fitness_scores))
            new_population.append(population[elite_idx])

            # Crossover dan Mutasi
            while len(new_population) < self.population_size:
                # Select parents
                parent1, parent2 = self.select_parents(population, fitness_scores)

                # Crossover
                if random.random() < 0.8:
                    point = random.randint(1, len(parent1) - 1)
                    child1 = parent1[:point] + parent2[point:]
                    child2 = parent2[:point] + parent1[point:]

                    if self.show_details and generation < 2:
                        print("\nCrossover:")
                        print(f"Parent 1: {self.format_schedule(parent1)}")
                        print(f"Parent 2: {self.format_schedule(parent2)}")
                        print(f"Titik crossover: {point}")
                        print(f"Child 1: {self.format_schedule(child1)}")
                        print(f"Child 2: {self.format_schedule(child2)}")

                else:
                    child1, child2 = parent1[:], parent2[:]

                # Mutasi
                for child in [child1, child2]:
                    if self.show_details and generation < 2:
                        print("\nMutasi:")
                        print(f"Sebelum: {self.format_schedule(child)}")

                    mutations = False
                    for i in range(len(child)):
                        if random.random() < 0.1:  # probabilitas mutasi
                            old_value = child[i]
                            child[i] = random.randint(0, 7)
                            mutations = True
                            if self.show_details and generation < 2:
                                print(f"Gen {self.activity_list[i]}: {old_value} -> {child[i]}")

                    if self.show_details and generation < 2:
                        if mutations:
                            print(f"Setelah: {self.format_schedule(child)}")
                        else:
                            print("Tidak ada mutasi")

                new_population.append(child1)
                new_population.append(child2)

            population = new_population[:self.population_size]

            # Tampilkan statistik generasi
            current_fitness = [self.calculate_fitness(ind) for ind in population]
            print(f"\nStatistik Generasi {generation + 1}: ")
            print(f"Rata-rata Fitness : {np.mean(current_fitness):.2f}")
            print(f"Fitness Tertinggi : {max(current_fitness):.2f}")
            print(f"Fitness Terendah  : {min(current_fitness):.2f}")

            if best_fitness >= 10:
                print(f"\n Solusi optimal ditemukan!")
                break

        print("\n Hasil Akhir")
        print(f"Jadwal Terbaik: {self.get_sequence(best_solution)}")
        print(f"Kromosom: {best_solution}")
        print(f"Fitness : {best_fitness:.2f}")

        print("\nJadwal Detail:")
        for i, activity in enumerate(self.activity_list):
            start_time = best_solution[i]
            end_time = start_time + self.activities[activity]
            print(f"Kegiatan {activity}: Mulai jam {start_time}:00. Selesai jam {end_time}:00")

        print("\nStatistik Evolusi:")
        print(f"Total Generasi : {generation + 1}")
        print(f"Peningkatan Fitness : {initial_fitness[elite_idx]:.2f} -> {best_fitness}")

        return best_solution, best_fitness

def main():
    print("Jadwal Aktivitas")
    print("Ketentuan 1 : A harus sebelum B")
    print("Ketentuan 2 : C dan D tidak boleh terjadi bersamaan")
    print("Ketentuan 3 : Total durasi tidak boleh melebihi 8 jam")
    print("\nDurasi kegiatan :")
    print("A : 2 jam")
    print("B : 3 jam")
    print("C : 2 jam")
    print("D : 4 jam")

    ga = Schedul(population_size=10, generations=10)
    best_solution, best_fitness = ga.run()

if __name__ == "__main__":
    main()


Jadwal Aktivitas
Ketentuan 1 : A harus sebelum B
Ketentuan 2 : C dan D tidak boleh terjadi bersamaan
Ketentuan 3 : Total durasi tidak boleh melebihi 8 jam

Durasi kegiatan :
A : 2 jam
B : 3 jam
C : 2 jam
D : 4 jam
Format: [Waktu mulai A, B, C, D] -> Fitness

Menampilkan 5 individu pertama dari 10 individu
Individu  1: [7, 3, 5, 6] -> Fitness: 4
          Jadwal: A:7 | B:3 | C:5 | D:6
          Urutan: ['B', 'C', 'D', 'A']
           Pelanggaran: A harus sebelum B, C dan D overlap, Melebihi 8 jam, A dan D overlap, B dan C overlap, C dan D overlap
Individu  2: [6, 3, 2, 0] -> Fitness: 5
          Jadwal: A:6 | B:3 | C:2 | D:0
          Urutan: ['D', 'C', 'B', 'A']
           Pelanggaran: A harus sebelum B, C dan D overlap, B dan C overlap, B dan D overlap, C dan D overlap
Individu  3: [7, 2, 5, 0] -> Fitness: 7
          Jadwal: A:7 | B:2 | C:5 | D:0
          Urutan: ['D', 'B', 'C', 'A']
           Pelanggaran: A harus sebelum B, Melebihi 8 jam, B dan D overlap
Individu  4: [6, 7, 0, 5]